# Super resolution

The objective of this practical session is to create a multi-frame super-resolution algorithm and apply it to real satellite images. 


We cover the following topics:
* Image interpolation and sampling
* Simulate realistic series of low-resolution frames
* Fuse the samples from a set of registered low-resolution images into a high resolution one
* Register a set of frames
* Application to real satellite images 


### Name: Mercier Marine 

#### Instructions
To solve this practical session, answer the questions below. Then, clear all the output cells using the menu option **Cell->All Output->Clear** and export the notebook with your answers using the menu option **File -> Download as -> Notebook (.ipynb)**. Then [submit the resulting file here](https://forms.gle/k6FSX2BrPix823dx6) by next week. You will receive an automatic acknowledgement of receipt.

This TP contains 6 exercices that are mainly focused on studying the properties of the presented algorithmic bricks.

In [ ]:
# Setup code for the notebook

# The following lines install the necessary packages in the colab environment
try:
    from google.colab import files

    # download TP data and tools
    !wget -q http://boucantrin.ovh.hw.ipol.im/static/facciolo/mvaisat/tp6.zip
    !unzip -q -o tp6.zip
    
    # install dependencies
    !apt-get -qq install libnfft3-dev
    !python -m pip -q install numpy matplotlib scipy geojson pyproj==2.4.1 opencv-contrib-python==4.5.3.56 rasterio  numba  pyfftw pyvoronoi rpcm
    !python -m pip -q install git+https://github.com/pyNFFT/pyNFFT.git@e2da0af374c6d7cc38992936ce4dda6e6c0ad7f1

except ImportError:
#    %matplotlib notebook
    pass

# Autoreload python modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import numpy as np                   # numeric linear algebra
import scipy
import scipy.ndimage                # only for ndimage.affine_transform
import matplotlib.pyplot as plt      # plotting
from numpy.fft import fft, ifft, fft2, ifft2, fftshift, ifftshift, fftfreq
from nfft_tools import initS, applyS, applySadj

# imports specific to this course
import utils          # IO and conversion tools (from TP-collection)
import vistools       # display tools (from TP-collection)
from sr import imshowfft, fft_translate, fft_zoom

Selecting previously unselected package libfftw3-long3:amd64.
(Reading database ... 155113 files and directories currently installed.)
Preparing to unpack .../0-libfftw3-long3_3.3.7-1_amd64.deb ...
Unpacking libfftw3-long3:amd64 (3.3.7-1) ...
Selecting previously unselected package libfftw3-quad3:amd64.
Preparing to unpack .../1-libfftw3-quad3_3.3.7-1_amd64.deb ...
Unpacking libfftw3-quad3:amd64 (3.3.7-1) ...
Selecting previously unselected package libfftw3-single3:amd64.
Preparing to unpack .../2-libfftw3-single3_3.3.7-1_amd64.deb ...
Unpacking libfftw3-single3:amd64 (3.3.7-1) ...
Selecting previously unselected package libfftw3-bin.
Preparing to unpack .../3-libfftw3-bin_3.3.7-1_amd64.deb ...
Unpacking libfftw3-bin (3.3.7-1) ...
Selecting previously unselected package libfftw3-dev:amd64.
Preparing to unpack .../4-libfftw3-dev_3.3.7-1_amd64.deb ...
Unpacking libfftw3-dev:amd64 (3.3.7-1) ...
Selecting previously unselected package libnfft3-double2:amd64.
Preparing to unpack .../5-libnf

# I. Image model, sampling and interpolation

Shannon's sampling theorem states that a band limited image $u$ can be **reconstructed exactly from its integer samples** via sinc interpolation
$$u(x) = \sum_{k\in\mathbb{Z}^2} u(k) \operatorname{sinc}(x-k).$$

For finite images of size $N^2$ the above formula leads to a trigonometric polynomial interpolation model 
$$u(x) = \sum_{\omega \in [-N/2...N/2-1]^2} \hat u(\omega) \exp ( -i 2\pi \omega x/N  ),$$
where  the coefficients $\hat u(\omega) $ are obtained as the discrete fourier transform (DFT) of the integer samples of $u$.


However, this interpolation method is not practical for numerical purposes because the sinc kernel is infinitely supported and the evaluation of the (finite) trigonometric polynomial involves all the $N^2$ samples of $\hat u(\omega)$. We can obtain an approximation by truncating this kernel, but we still need a very large kernel for good accuracy.

There are two alternatives to obtain good accuracy:
1. Use properties of the Fourier transform to obtain exact interpolations in particular cases
1. Use compactly supported kernels that are good approximations of the sinc kernel combined with exact interpolation.

In [ ]:
# the input image u
u = utils.readGTIFF('data/barb.png').squeeze()[100:356,100:356]
sh = u.shape
# show the image and its fourier transform
imshowfft( u )

### Fourier interpolation

Assume we want to obtain the interpolated values $u(x)$ for $x$ on a grid $\Gamma = T(\mathbb{Z}^2)$ which is a transformation of the original sampling grid. When $T$ has a particular structure we can use some properties of the Fourier transform to compute the exact interpolation

#### Case 1:  $T$ is a translation

If $T(x) = x+t$ for some $t\in\mathbb{R}^2$ then we can use the following property of the Fourier transform

$ \mathcal{F}(u \circ T)(\omega) = e^{2\pi i \omega t} \mathcal{F}(u)(\omega) $

The function `fft_translate` uses this property and the FFT algorithm to translate an image by a given amount.

In [ ]:
def fft_translate_v0(im, dx, dy):
    '''
    Subpixel translation using Fourier interpolation
    
    Args:
        im (np.ndarray): numpy array containing the input image
        dx,dy (floats): subpixel shift to be applied
    Returns:
        np.ndarray: containing the resampled shifted image 
    '''
    import pyfftw
    import numpy as np

    imF = np.fft.fft2(im)
    X, Y = np.meshgrid(np.fft.fftfreq(im.shape[1]), np.fft.fftfreq(im.shape[0]))
    phi =  2j*np.pi*( X*dx + Y*dy ) 
    imFshift = imF * np.exp(-phi)
    return  np.real(np.fft.ifft2(imFshift))

In [ ]:
# here we test the translation on an image 
dx,dy = 10.3, 8.8
ut = fft_translate_v0(u, dx,dy)

vistools.display_gallery([ut, u],['translated by: %g,%g'%(dx,dy), 'original'])

Note that the image is correctly translated, however the periodicity assumption of the DFT leads to ringing effects at the image boundaries. 

Nevertheless, the representation is exact and we should be able to get back to the initial image. 

> **<u>Excercise 1</u>**: Check that `fft_translate` works correctly. Choose two **NON INTEGER** translations $t_1$ and $t_2$ such that $t_1 + t_2 = 0$. Translate an image by $t_1$ then by $t_2$. Compare the result to image $u$ and check the error with respect to the translated image.
>
> **<u>Bonus</u>**: We could keep translating the image by non-integer steps back and forth without ever loosing precision. Try doing 100 translations with `fft_translate` and compare with bilinear or bicubinc interoplation (which are implemented in `scipy.ndimage.map_coordinates`).

In [ ]:
### EXERCISE 1. WRITE YOUR SOLUTION CODE HERE*
from scipy import ndimage
t1 = [3.8, 2.7] 
ut = u.copy()
vt = u.copy()

for i in range(100):
    ut = fft_translate_v0(ut, t1[0], t1[1])
    ut = fft_translate_v0(ut, -t1[0], -t1[1])

    vt = ndimage.interpolation.map_coordinates(vt, mode='nearest')

vistools.display_gallery([ut, u],['result images', 'original'])

In practice the rebounds resulting from the periodization are a nuance, and there are different ways to suppress them.

- One consists in symmetrizing the image before applying the transformation (which is implemented in `sr.fft_translate`). 

- Another option is to smooth the image close to the image boundaries so that there are no discontinuities in the periodic image (see [Moisan 2011](http://helios.mi.parisdescartes.fr/~moisan/papers/download.php?file=2009-11r.pdf) for instance).

In [ ]:
dx,dy = 10.3, 8.8
ut = fft_translate(u, dx,dy)

vistools.display_gallery([u,ut],['original','translated by: %g,%g'%(dx,dy)])

#### Case 2: $T$ is a zoom

If $T(x) = x/z$ for some zoom factor $z\in\mathbb{Z}$ then we can obtain a zoomed version of $u$ by zero padding.  The  function `fft_zoom` uses this property and the FFT algorithm to obtain a zoomed version of the input image.

> **<u>Excercise 2</u>**: Check that `fft_zoom` works correctly. You will perform two checks:
>
> 1. Compute a x2 zoom and check that the even pixels of the zoomed image coincide with the pixels of the original image
> 2. Visualize the spectrum of the zoomed-in image in log scale (only the central part of the spectrum should be non-zero). You can use `imshowfft` function for this visualization.

In [ ]:
def fft_zoom(image, z=2):
    """
    Zoom an image by zero-padding its Discrete Fourier transform.
    
    Args:
        image (np.ndarray): 2D grid of pixel values.
        z (int): Factor by which to multiply the dimensions of the image. Must be >= 1.
    Returns:
        np.ndarray: zoomed image.
    """
    h, w = image.shape

    # zero padding sizes
    padw = (z - 1) * w
    padh = (z - 1) * h
    
    # Fourier transform with the zero-frequency at 0,0
    ft = np.fft.fft2(image)

    # the zoom-in is performed by zero padding the Fourier transform
    ft = np.hstack((ft[:,:w//2], np.zeros((h,padw))     , ft[:,w//2:]))    
    ft = np.vstack((ft[:h//2,:], np.zeros((padh,w+padw)), ft[h//2:,:]))    

    # take the inverse Fourier transform
    out = np.fft.ifft2(ft)

    # if the input is a real-valued image, then keep only the real part
    if np.isrealobj(image):
        out = np.real(out)

    # to preserve the values of the original samples, the L2 norm has to be
    # multiplied by z*z.
    return out * z * z

In [ ]:
### EXERCISE 2. WRITE YOUR SOLUTION CODE HERE

###   B-spline + FFT interpolation = NFFT

A common alternative to Fourier-based interpolation is the family of B-spline interpolators of different orders.  When the order of the spline tends to infinity spline interpolation converges to sinc interpolation. However increasing the spline order also increases the computational cost because the support of the kernel is proportional to the spline order (squared in dimension 2).

A better computation/accuracy tradeoff can be obtained by combining a zoom by fft up to a fixed zoom level and B-spline interpolation of a moderate order on the zoomed image.


**The nonuniform FFT library (NFFT)** uses these techniques to implement efficient and precise trigonometric polynomial evaluation functions. 

Conceptually, NFFT can be seen as implementing a linear operator $S$ of size $M\times N$, where $N$ is the number of coefficients in the trigonometric polynomial, and $M$ is the number of irregularly sampled the pixels at positions $\xi_i$
$$((S_{ij}))  =  exp( -i 2\pi \xi_i j/N).$$

Applying $S$ to a Fourier transform $\hat u$ evaluates the trigonomatric polynomial on the samples:  $u(\xi_i) = \sum_j  S_{ij}  \hat u_j$.

There are many nuances in the NFFT, which are incapsulated in the module `nfft_tools`. The two main functions exposed by this module are `initS` and `applyS` which define and apply the sampling operator $S$. 


```python

    def initS(samples, N, nfft_m=9):
        """
        prepare the NFFT plan for the samples
        Args:
            samples: array of size (M,2) with the 2D coordinates (in the interval [0,1]^2) of the M samples
            N      : is a 2-element array specifying the 2D bandwidth or the order of the trigonometric polynomial
            nfft_m : is an internal parameter controlling the quality of the sampling 
        Returns:
            the NFFT plan needed to apply the transformation on a data vector
        """


    def applyS(fhat, p):
        """
        Samples the trigonometric polynomial fhat using NFFT
        Args:
            fhat: is the regular 2D fft of an image **(but fftshifted and conjugated)**, 
            p   : is the plan specifying the positions of the samples to be computed 
        Returns:
            the values sampled at the positions indicated in the plan
        """

 ```

The following blocks use these functions first to verify the consistency of NFFT and FFT, and then to sample the trigonometric polynomial associated to an image.

#### Sanity check: sampling the trigonometric polynomial at the regular positions yields the same image as the IFFT

Since the NFFT S operator is computing an approximate IFFT it should coincide with it. 

In [ ]:
from numpy.fft import fftshift, ifftshift, fft2, ifft2
from nfft_tools import initS, applyS, applySadj, initT, applyT
import numpy as np



u = utils.readGTIFF('data/barb.png')[:,:,0].squeeze()#.astype(np.double)[:250,:257]

imshowfft(u)


# shifted conjugated fft is the representation of nfft 
fft_u = fft2(u) 
fft_u2 = fftshift(fft_u).conj()   


# regular grid size 
N = u.shape

# irregular sample  positions
xx, yy = np.meshgrid( np.arange(N[1], dtype=np.double)/(N[1]) ,  np.arange(N[0], dtype=np.double)/(N[0]) )

samples = np.array(list (zip(yy.flatten(), xx.flatten()))  )
M = len( xx.flatten() )   # number of samples 

## output: xx, yy, samples, M, shape

print(N,M)

# initialize the nfft with this samplig grid
p = initS(samples, N)

# compute the inverse fft using nfft
irreg = applyS(fft_u2,p)
u2 = np.real(irreg.reshape((N[0],N[1])) )

# check the difference of the reconstruction 
plt.figure()
plt.imshow( u - u2, vmin=-1,vmax=1 )
plt.title('difference')
plt.colorbar()

#### Sample an image on an irregular grid (here perturbed)

Here we show how NFFT allows to efficiently sample the trigonometric polynomial of an image without restrictions on the shape of the sampling grid. 

In [ ]:
# regular grid size 
N = u.shape

# irregular sample positions: a smoothly perturbed grid
xx, yy = np.meshgrid( np.arange(N[1], dtype=np.double)/(N[1]) ,  np.arange(N[0], dtype=np.double)/(N[0]) )
xx =  xx + 1*scipy.ndimage.gaussian_filter(np.random.randn(*xx.shape)/4, 8)
yy =  yy + 1*scipy.ndimage.gaussian_filter(np.random.randn(*yy.shape)/4, 8)

samples = np.array(list (zip(yy.flatten(), xx.flatten()))  )
M = len( xx.flatten() )   # number of samples 

fft_u2 = fftshift(fft2(u)).conj()   # shifted conjugated fft is the representation of nfft 

# init nfft
p = initS(samples, N)

irreg = applyS(fft_u2,p)

# show the irregular samples
imshowfft(  np.real(irreg.reshape((N[0],N[1])) ) , title='image sampled on a perturbed grid' )

# II. Image simulation

In this section we will simulate low resolution images as aquired by the Skysat satellite. 
We consider the following simplified image formation model 

$$ v =  \Delta   ( T  ( K_{\text{diffraction}} \ast K_{\text{sensor}}\ast u )  ) + \text{noise},$$ 

where $u$ is an ideal image (of infinite resolution), $K_{\text{diffraction}}$ is the PSF due to the diffraction,   $K_{\text{sensor}}$ the PSF due to the sensor integration, $T$ denotes a spatial transformation, and $\Delta$ denotes the sampling step. Note that in this formulation the spatial transformation $T$ was commuted with convolutions and in general this is not correct. However, for the case of remote sensing the transformation $T$ is almost a rigid transformation in  which case the commutation is correct.  

Since convolutions are easily modeled in the frequency domain we instead manipulate the OTF (optical transfer functions) associated to our kernels: $H_{\text{diffraction}}$, and $H_{\text{sensor}}$.

In order to correctly represent the alias resulting from the sampling we shall simulate our images at an oversampled rate controlled by the zoom factor $z=2$. This "zoomed" image will be our infinite resolution image. For simplicity, our input image will be squared and of size $N\times N$. 
It is important to note that the pixels of this oversampled image are therefore of size $3.25µm \times 3.25µm$: half the size of the target resolution. 

These are the relevant parameters to be considered (in cm)

    D = 35               #cm             # circular aperture diameter 
    f = 360              #cm = 3.6m      # focal lenght 
    lam = 6e-5           #cm = 0.6µm     # visible light wavelenght (green)
    pixel_pitch = 6.5e-4 #cm = 6.5µm     # spacing of the pixels (pixel area is slightly smaller)
    alt = 5e7            #cm = 500km     # altitude of the satellite

In [ ]:
# skysat's parameters 

D=35      #cm 
f=360     #cm = 3.6 m
lam=6e-5  #cm = 0.6 micrometers   # ~visible light 
pixel_pitch = 6.5e-4 #cm   6.5 µm
photodiode_ratio=0.8 #assumed relative size of the photodiode wrt the pixel_pitch

### Sensor integration OTF

<img width=500px src="https://d1d1c1tnh6i0t6.cloudfront.net/wp-content/uploads/2017/08/pixel-size-2.jpg">

The OTF associated to the sensor integration is defined relative to the sampling step.
Assuming a squared integration window the 2D OTF is the sinus cardinal $sinc(\omega_x/2)sinc(\omega_y/2)$.

Given that the photodiode area is just a part of the surface of the pixel we should consider an integration window that is smaller than the sampling step, this amounts to changing the width of the $sinc$. 

Let us start by computing and displaying the $sinc$ profile, zoomed in order to see its effects on the higher frequencies.

In [ ]:
# the input image u
u = utils.readGTIFF('data/barb.png').squeeze()[100:356,100:356]
sh = u.shape

In [ ]:
def sinc_OTF_profile(N, zoom, photodiode_ratio=1):
    '''
    Generates the zoomed version of the 1D sensor OTF profile
    Args:
        N: is the lenght of the signal to be produced
        zoom: specifies the oversampling factor. zoom=2 yields the OTF at 2 x Nyquist
        photodiode_ratio (default 1): is the size of the pixel relative to the separation
    Returns:
        a vector w with the sampled frequencies 
        and a vector with the corresponding OTF values
    '''
    # frequency range [-z*pi, z*pi]
    #w = np.arange(-N/2,N/2)*zoom*np.pi*2/N
    w = fftshift(fftfreq(N,d=1/zoom))*np.pi*2
    
    # define the unnormalized sinc
    sinc = lambda w : np.sinc(w/np.pi)  
    
    return w, sinc(w/2*photodiode_ratio)
        

# display zoom factor
z = 2

# relative size of the photodiode (usually < 1) 
photodiode_ratio=0.8

w, Hsensor = sinc_OTF_profile(sh[0], z, photodiode_ratio)

plt.figure()
plt.grid('on')
plt.title('frequency profile of the OTF')
plt.vlines([-np.pi,np.pi], 0, 1) # Nyquist
plt.plot(w, Hsensor)
plt.show()

Then we can use this profile to generate the OTF for the 2D sensor. 
We are interested in generating a zoomed version of the OTF so that it could be applied on the large image to simulate the sensor.

In [ ]:
def sensor_OTF(sh,zoom,photodiode_ratio=1.0):
    '''
    Generates the zoomed version of the sensor OTF 
    Args:
        sh: 2-element array with the shape [rows, cols] of the image to be generated
        zoom: the oversampling factor to be considered 
        photodiode_ratio (default 1): is the size of the pixel relative to the separation
    Returns:
        the sensor OTF with the 0 shifted at coordinate at 0,0
    '''
    
    _, Hcol = sinc_OTF_profile(sh[0], zoom, photodiode_ratio)
    _, Hrow = sinc_OTF_profile(sh[1], zoom, photodiode_ratio)
    
    sinc = lambda w : np.sinc(w/np.pi)  # unnormalized sinc

    H_sensor = (Hcol[np.newaxis,:]).transpose() @ Hrow[np.newaxis,:]
    return ifftshift(H_sensor)   # put the zero frequency at it's place
    

    
# oversampling factor "zoom"
z = 2

# relative size of the photodiode (usually < 1) 
photodiode_ratio=0.8
     
H_sensor = sensor_OTF(sh,z,photodiode_ratio)
        
plt.figure()
plt.title('sensor OTF, 2x ')
plt.imshow( fftshift(H_sensor) )
plt.colorbar()
plt.show()

### Diffraction OTF (circular aperture) 


A circular aperture with diameter $D$ produces a point-spread-function due to diffraction  called Airy disk 

<img width=150 src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Airy_disk_D65.png/220px-Airy_disk_D65.png">


Its OTF decreases with increasing spatial frequency and vanishes at the diffraction limit. 
<img width=400 src="https://i0.wp.com/www.strollswithmydog.com/wordpress/wp-content/uploads/Sampled-DiffSquare-with-Slice.png">

The diffraction limit  is given by the formula 
$$ \rho_c = \frac{2\pi D }{\lambda f}   \quad [cycles/cm],  $$ 
where $\lambda$ is the wavelength of light and $f$ is the focal lenght. 

This cutoff frequency corresponds to a samplig step equal to 
$$ \delta_c = \pi / \rho_c = \frac{\lambda f}{2 D} \quad [cm].$$

The diffraction OTF profile is not exactly linear, its  form is given by the following formula which is normalized from 0 to the frequency cutoff 
$$ H_{\text{diffraction}}(\omega) =  \frac{2}{\pi}  \left( \arccos(\rho(\omega)) -  \rho(\omega)*\sqrt{1-\rho(w)^2} \right)  \quad \text{with} \quad \rho(\omega) = \omega / \rho_c.$$

In [ ]:
def diffraction_cutoff_frequency(D,f,lam):
    """
    Computes the diffraction_cutoff_frequency for a circular aperture
    Args:
        D: diameter of the aperture
        f: focal lenght 
        lam: wavelength of the light 
    All arguments must be expressed on the same linear units e.g. cm
    Returns:
        the curoff frequency expressed in cycles/cm, where a cycle is 2π
    """
    return (2*np.pi*D)/(lam*f)

    
# difraction cutoff frequency 
cutoff_freq = diffraction_cutoff_frequency(D,f,lam)
print('the diffraction cutoff frequency is:', cutoff_freq, 'cycles/cm')

# the corresponding cycles per pixel  
cutoff_freq_px = cutoff_freq/np.pi * pixel_pitch    #
print('which amounts to:', cutoff_freq_px,   'cycles/px')

In [ ]:
def diffraction_OTF_profile(w):
    """
    Evaluates the 1D normalized diffraction OTF 
    the diffraction cutoff is attained when w is 1, beyond that value 0 is returned
    Args: 
        w: the normalized frequency 
    Returns: 
        the normalized diffraction OTF
    """
    w = np.abs(w).clip(0,1)
    return   np.where(w<1, 2/np.pi* ( np.arccos(w) -   w*np.sqrt(1-w**2)) , 0)


# frequency range of the zoomed image [-z*pi, z*pi]
#w = np.arange(-N/2,N/2)*2/N*z*np.pi
w = fftshift(fftfreq(sh[0],d=1.0/z))*np.pi*2
# normalized difraction OTF profile 
H_diff = diffraction_OTF_profile(w/np.pi)

plt.figure()
plt.grid('on')
plt.title('normalized difraction OTF profile up to 2x the diffraction cutoff')
plt.plot(w, H_diff)
plt.show()

We need to compare the spatial frequency of the our simulated sensor with the limit of the diffraction kernel

In [ ]:
print('The pixel step is:',  pixel_pitch, 'cm', '(or', 1/pixel_pitch, 'samples/cm)' )
print('Given that the target pixel step is', pixel_pitch , 'cm, our zoomed image has a pixel of:', pixel_pitch/z, 'cm')
print('')

max_frequency_zoomed_image = np.pi/ (pixel_pitch/z)
print('Thus the maximum spatial frequency visible with our zoomed pixels is', max_frequency_zoomed_image,'cycles/cm')
print('while the spatial frequency of the diffraction cutoff  is', cutoff_freq,'cycles/cm')
print('')


relative_cutoff = cutoff_freq/max_frequency_zoomed_image
print('In conclusion, the diffraction cutoff frequency is', relative_cutoff, 'times the one of the zoomed image' )

We  can now simulate the diffraction OTF

In [ ]:
def diffraction_OTF(sh,D,f,lam,pixel_pitch,zoom):
    '''
    Generates the diffraction OTF for a zoomed image, simulating a specific pupil diameter D, focal lenght f, and pixel_pitch
    Args:
        sh: 2-element array with the shape [rows, cols] of the image to be generated      
        D: diameter of the aperture
        f: focal lenght 
        lam: wavelength of the light to be used 
        zoom: the oversampling factor to be considered 
        
    D,f,lam,pixel_pitch: must be expressed on the same linear units e.g. cm

    Returns:
        the diffraction OTF with the 0 shifted at coordinate at 0,0
    
    '''
    
    # difraction cutoff frequency 
    cutoff_freq =  diffraction_cutoff_frequency(D,f,lam)

    # sensor cutoff frequency
    max_frequency_zoomed_image = np.pi/ (pixel_pitch/zoom)

    relative_cutoff = cutoff_freq/max_frequency_zoomed_image
    
    # the frequency range of the output image [-z*pi, z*pi]^2
    #wx = np.arange(-sh[1]/2,sh[1]/2)*zoom*np.pi*2/sh[1]
    #wy = np.arange(-sh[0]/2,sh[0]/2)*zoom*np.pi*2/sh[0]
    wx = fftshift(fftfreq(sh[1],d=1.0/zoom))*np.pi*2
    wy = fftshift(fftfreq(sh[0],d=1.0/zoom))*np.pi*2
    X,Y = np.meshgrid(wx,wy)

    # diffraction profile 
    H = diffraction_OTF_profile( np.hypot(X,Y)/ (zoom*np.pi) /relative_cutoff)
          
    return  ifftshift( H )

  
H_diffraction  = diffraction_OTF(sh,D,f,lam,pixel_pitch,2)

plt.title('diffraction OTF, 2x')
plt.imshow( fftshift( H_diffraction) )
plt.show()

### Simulation of the system


Putting together the previous steps we can simulate our sysmtem:

$$ v =  \Delta   ( T  ( K_{\text{diffraction}} \ast K_{\text{sensor}}\ast u )  ) + \text{noise}$$ 

1. take a high resolution image and a fix a zoom factor (the simulated image will be z times smaller)
2. generate the high resolution sensor and diffraction OTF for corresponding to the current system
3. apply the convolution ot the high resolution image
4. generate a sampling grid and sample the high resolution image (here we will only consider grids that correspond to translations)

In [ ]:
# skysat's parameters 

D=35      #cm 
f=360     #cm = 3.6 m
lam=6e-5  #cm = 0.6 micrometers   # ~visible light 
pixel_pitch = 6.5e-4 #cm   6.5 µm
alt = 50000000 # 500km
photodiode_ratio=.8  # not the actual value 
z=2       # zoom factor of the high resolution image

In [ ]:
def simulate_LR(u, H, dx, dy, z = 2, sigma=0):
    """
    Simulate a low resolution image from the high resolution image 
        v = Pi_z (  T_{dx,dy} ( H*u ) ) + N(sigma,0) 
    Args:
        u: input high resolution image
        H: applying the OTF
        dx,dy: shifts the sampling grid by (dx,dy)
        z: integer subsampling factor  (default 2) 
        sigma: std of the noise to be added to the samples 
    Returns:
        the simulated image, 
        the X,Y coordinates of the samples, 
        and a binary mask indicating if the samples are invalid
    """
    
    from sr import fft_translate
    h, w = u.shape

    # precompute the positions of the samples 
    X, Y = np.meshgrid(np.arange(w).astype(float), np.arange(h).astype(float))

    # apply the OTF  
    u_bl = np.real(ifft2(fft2(u) * H))

    # translate and subsample 
    u_lr = fft_translate(u_bl, dx, dy)[::z,::z]
    # do the same with the sample positions 
    Xs = (X[::z,::z]-dx)/2
    Ys = (Y[::z,::z]-dy)/2
    
    # add noise 
    u_lr +=  np.random.randn(*u_lr.shape)*sigma
    
    
    # determine mask 
    Ms =  ((Xs[0,:]>0)  *  (Xs[0,:] <= w) *  (Ys[1,:] >0) * (Ys[1,:] <= h) )
    
    
    return u_lr,Xs,Ys,Ms


sh = u.shape
dx,dy = 0.3,0.6
# generate the OTF 
H_sensor = sensor_OTF(sh,z,photodiode_ratio=photodiode_ratio)
H_diffraction =  diffraction_OTF(sh,D,f,lam,pixel_pitch,z)

# plot the profile of the simulated OTF 
plt.figure()
plt.vlines([sh[1]/4,3*sh[1]/4], 0, 1) # Nyquist
plt.plot(  fftshift((H_sensor * H_diffraction)[0,:] ) )


# simulate the LR image 
v, Xs, Ys, Ms = simulate_LR(u, H_sensor * H_diffraction, dx, dy)

# high resolution but blurred image
u_bl = ifft2(fft2(u)* H_sensor  * H_diffraction )
imshowfft( u_bl ,z, title='high resolution filtered' )


# subsampled image
imshowfft( v , title='subsampled')

# III. Fusion

The fusion steps takes a set of aliased images (that are aligned on the same reference)  and fuse the samples to produce a higher resolution image.

We first produce a set of images using the simulator from the previous section. Then we will compare two fusion techniques:
* Shift-and-Add, which accumulates the samples of the images to the nearest neighbor of the high resolution image and
* ACT, which fits a trigonometric polynomial model to the samples

### Generate a sequence of LR frames

In [ ]:
def simulate_LR_stack_sanity_check(u, H):
    """
    This function simulates only one sampling with 4 integer positions 
    """
    ## sanity check ! 
    shifts = np.array([[0,0],[0,-1],[-1,0],[-1,-1]])

    # generate the stack 
    Xs = []
    Ys = []
    Vs = []
    Ms = []
    for i,(dx,dy) in enumerate(shifts) :
        print(i,dx,dy)
        V,X,Y,M = simulate_LR(u, H, dx, dy, z=2, sigma=0)

        # refer the shifts are already referred to the subsampled images
        Xs.append(X)
        Ys.append(Y)
        Vs.append(V)
        Ms.append(M)

    Xs=np.array(Xs)  
    Ys=np.array(Ys)  
    Vs=np.array(Vs)  
    Ms=np.array(Ms)  

    return Vs,Xs,Ys,Ms



def simulate_LR_stack(u, H, nframes, z=2, sigma=0, jitter=0):
    """
    """
    
    ## random shifts forcing the first one to be 0,0
    shifts = np.random.rand(nframes,2)*4 -2
    shifts[0] = [0,0]

    # generate the stack 
    Xs = []
    Ys = []
    Vs = []
    Ms = []
    for i,(dx,dy) in enumerate(shifts) :
        #print(i,dx,dy)
        V,X,Y,M = simulate_LR(u, H, dx, dy, z=z, sigma=sigma)
        # add jitter
        X += np.random.randn(1)*jitter
        Y += np.random.randn(1)*jitter
        # refer the shifts are already referred to the subsampled images
        Xs.append(X)
        Ys.append(Y)
        Vs.append(V)
        Ms.append(M)

    Xs=np.array(Xs)  
    Ys=np.array(Ys)  
    Vs=np.array(Vs)  
    Ms=np.array(Ms)  

    return Vs,Xs,Ys,Ms

In [ ]:
# the input image u
u = utils.readGTIFF('data/barb.png').squeeze()[100:356,100:356]
sh = u.shape

H_sensor      = sensor_OTF(sh,z,photodiode_ratio=photodiode_ratio)
H_diffraction =  diffraction_OTF(sh,D,f,lam,pixel_pitch,z)
Vs,Xs,Ys,Ms = simulate_LR_stack(u, H_sensor*H_diffraction, 10, z = 2, sigma=0)

vistools.display_gallery(Vs)

### Shift-and-Add 

The shift-and-add accumulates the samples of the images to the nearest neighbor of the high resolution image. This process might leave some pixels without votes, so a last postprocess fills the missing pixels with the median of its neighbors.

In [ ]:
def shift_and_add(outshape, Vs, Xs, Ys, Ms=None, fill_missing=False):
    """
    Apply the shift-and-add fusion algorithm
    Args:
        outshape: 2-element array with the size of the output image
        Vs: array containing the sample values
        Xs: array containing the x pixel coordinate of the samples 
        Ys: array containing the y pixel coordinate of the samples 
        Ms: array containing the masks of valid samples 
    Returns:
        np.ndarray: of size outshape with the resulting image
    """
  
    if Ms is None:
        Ms = np.ones_like(Ys)
    Ms = Ms > 0  # make it bool

    h, w, = outshape
    acc  = np.zeros((h,w))
    count = np.zeros((h,w))
    
    # accumulate the pixels of each image Vs[i] on acc and count
    for i in range(Vs.shape[0]):
        Mp = Ms[i]
        Vp = Vs[i][Mp]
        Xp = Xs[i][Mp]
        Yp = Ys[i][Mp]
        
        Xp = (np.clip(np.round(Xp),0,w-1)).astype(int)
        Yp = (np.clip(np.round(Yp),0,h-1)).astype(int)
        
        acc[Yp,Xp]   += Vp
        count[Yp,Xp] += np.ones_like(Vp)
        
    # catch the missing pixels 
    missing = np.where(count==0)
    count[missing] = 1
    
    # compute averages
    out = acc/count
    
    # fill-in the gaps with a median filter 
    if fill_missing and np.sum(missing)>0:
        #out[missing] = scipy.ndimage.median_filter(out, 3)[missing]
        ### Ideally one should ignore the missing pixels in the median
        ### but the following lines do not seem to work in all platforms
        out[missing] = np.nan
        out[missing] = scipy.ndimage.generic_filter(out, np.nanmedian, 3)[missing]

    return out

In [ ]:
h,w = Vs[0].shape
zz  = 2
u_sa = shift_and_add( [int(h*zz),int(w*zz)], Vs, Xs*zz, Ys*zz, Ms)

imshowfft(u_sa, title='S&A', z=zz)
imshowfft(u_bl, title='ideal' )


plt.figure()
plt.imshow(u_sa-np.real(u_bl) ,vmin=-10, vmax=10);plt.title('difference'); plt.colorbar()
vistools.display_gallery([u_sa, u_bl])

### The ACT algorithm


The ACT algorithm (Adaptive-weights Conjugate-gradient on Toeplitz-matrix) proposes to find the coefficients of a trigonometric polynomial $\hat u$ that solve the variational problem
$$ \min_{\hat u} \|S \hat u  -  z \|^2,$$ 
where $S$ denotes the sampling operator and $z$ are the observed samples. 
This yields the normal equations  $ S^* S \hat u = S^* z $  which can be solved by conjugate gradient. 

Adding a regularization term of the form $ \lambda \| D \hat u \|^2$, where $D$ can be a differential operator (e.g. the gradient), which is diagonal in the frequency domain, leads a similar linear system
$$ S^* S \hat u  + \lambda D^* D  \hat u  = S^* z.$$


The particularity of the ACT algorithm is that it exploits a property of $S^* S$, which is Toeplitz, to accelerate its evaluation. For sake of simplicity, here we show a simpler version that use the $S^*$ and $S$ at each iteration. The implementation using the Toeplitz trick is available in the module `sr`.

In [ ]:
from nfft_tools import initS, applyS, applySadj
from scipy.sparse.linalg import LinearOperator, cg 
from numpy.fft import fftshift, ifftshift, fft2, ifft2
import scipy.sparse.linalg
import numpy as np



def ACT_nfft(samples, irreg, N, maxiter=10, lam=0):
    
    #  regularization penalty in the frequency domain
    DDx, DDy = np.meshgrid( np.pi*  (np.arange(N[1], dtype=np.double) - N[1]//2 )/(N[1]) ,  
                            np.pi*  (np.arange(N[0], dtype=np.double) - N[0]//2 )/(N[0]) )
    LAP = lam * (DDx**2 + DDy**2)


    # initialize NFFT
    nfftplan = initS(samples, N)
    
    # apply adjoint sampling to compute b
    b = applySadj(irreg ,nfftplan).flatten()

    
    # prepare the linear operator  S^* S (which is Toeplitz)
    def StS(v):
        t0 = v.astype(np.complex).reshape(N)
        t1 = applyS(t0,nfftplan) 
        t2 = applySadj(t1,nfftplan)
        if lam > 0:
            t2 +=  LAP * t0
        return t2.flatten()
    A = LinearOperator((N[0]*N[1],N[0]*N[1]), matvec=StS, dtype=np.complex)

    
    # prepare the CG to solve Ax=b 
    def cb(x):
        cb.cgiter += 1
        if cb.cgiter % 10 == 0:
            print('cg iterations... %d'%cb.cgiter)
    cb.cgiter = 0
    x = scipy.sparse.linalg.cg(A, b, maxiter=maxiter, callback=cb)[0]

    tmp = x.reshape((N[0],N[1])) 

    tmp =  np.real(ifft2 (fftshift(tmp.conj())))

    return np.real(tmp)



def ACT(outshape, Vs, Xs, Ys, Ms=None):
    """
    Apply the ACT fusion algorithm (simplified interface)
    Args:
        outshape: 2-element array with the size of the output image
        Vs: array containing the sample values
        Xs: array containing the x pixel coordinate of the samples 
        Ys: array containing the y pixel coordinate of the samples 
        Ms: array containing the masks of valid samples 
    Returns:
        np.ndarray: of size outshape with the resulting image
    """
    h,w = outshape
    
    if Ms is None:
        Ms = np.ones_like(Ys)
    Ms = (Ms > 0)  # make it bool
    
    samplesloc =  np.vstack(( np.array(Ys[Ms]).flatten()/h  ,  np.array(Xs[Ms]).flatten()/w  )).T
    irregVal = np.array(Vs[Ms]).flatten()
    
    out = ACT_nfft(samplesloc, irregVal, outshape, maxiter=30, lam=0.01)
    return out

In [ ]:
h,w = Vs[0].shape
zz  = 2
u_act = ACT( [int(h*zz),int(w*zz)], Vs, Xs*zz, Ys*zz)

imshowfft(u_act, title='ACT', z=2)
imshowfft(u_bl, title='ideal image' )

plt.figure()
plt.imshow(u_act-np.real(u_bl) ,vmin=-10, vmax=10); plt.title('difference'); plt.colorbar()

Now that we've seen that ACT works, let's use its faster version from the `sr` module.

In [ ]:
from sr import ACT_nfft

### Deconvolution / sharpening

Since we restore the image convolved with the diffraction and sensor kernels it is reasonable that it looks a bit blurry. The following block applies a simple sharpening with the objective of improving the contrast.

In [ ]:
#u_act = ACT( [h*zz,w*zz], Vs, Xs*zz, Ys*zz)
def sharpen(u, alpha=1):
    """
    sharpen the image u by adding the discrete laplacian
    """
    import scipy.signal
    return u + alpha * scipy.signal.convolve (u, np.array([[1,-2,1],[-2,4,-2],[1,-2,1]])/4 ,mode='same' ) 
    
u_act1 = sharpen(u_act)

u_zp = fft_zoom(Vs[0],2)

vistools.display_gallery( [ utils.simple_equalization_8bit(x) for x in [u_act, u_act1, u_bl, u_zp] ] , ['ACT','ACT sharpened', 'filtered input', 'zero padded'])
imshowfft(utils.simple_equalization_8bit(u_act1), figsize=(20,10))

# IV. Registration


The registration 

* load the frames 
* prefilter the frames for better alignement 
* generate a sequence of homographies that register theimagesiiiiiiihow the aligned stack of images



We provide three real sequences: 

        files = glob("data/mire/crop*.tif")
        files = glob("data/buildings/smallim_*.tiff")
        files = glob("data/classic/smallim_*.tiff")

In [ ]:
def ECC_frame_align(im1_gray, im2_gray, gaussFiltSize=1):
    """
    Compute a parametric registration (homography) of image2 over image1 using the ECC algorithm
    Adapted from: https://www.geeksforgeeks.org/image-registration-using-opencv-python/
    Args: 
        im1_gray: target image 
        im2_gray: image to be registered 
        gaussFiltSize: (int>1) size of the gaussian prefilter 
    Returns: 
        a resampled version of im2_gray registered over im1_gray
        the 3x3 matrix describing the transformation     
    """
    import cv2
    import scipy.ndimage
    import numpy as np

    # Find size of image1
    sz = im1_gray.shape

    # Define the motion model
    warp_mode = cv2.MOTION_TRANSLATION
    warp_mode = cv2.MOTION_AFFINE
    warp_mode = cv2.MOTION_HOMOGRAPHY

    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else :
        warp_matrix = np.eye(2, 3, dtype=np.float32)

    # Specify the number of iterations.
    number_of_iterations = 30;

    # Specify the threshold of the increment
    # in the correlation coefficient between two iterations
    termination_eps = 1e-10;

    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

    # Prefilter the images to remove alias
    im1_grayB = scipy.ndimage.gaussian_filter(im1_gray.astype(np.float32), gaussFiltSize)
    im2_grayB = scipy.ndimage.gaussian_filter(im2_gray.astype(np.float32), gaussFiltSize)
    
    # Run the ECC algorithm. The results are stored in warp_matrix.
    (cc, warp_matrix) = cv2.findTransformECC(im1_grayB, im2_grayB,
                                             warp_matrix, warp_mode, 
                                             criteria, None)

    # just convert all warp matrices into Homographies
    if warp_matrix.shape[0] == 2:
        warp_matrix = np.vstack([warp_matrix,[0,0,1]])
    
    # Use warpPerspective for Homography
    im2_gray_aligned = cv2.warpPerspective (im2_gray, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LANCZOS4 + cv2.WARP_INVERSE_MAP)
    ## Use warpAffine for Translation, Euclidean and Affine
    #im2_gray_aligned = cv2.warpAffine(im2_gray, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LANCZOS4 + cv2.WARP_INVERSE_MAP);
        
    return im2_gray_aligned, warp_matrix

In [ ]:
from glob import glob

files = glob("data/mire/crop*.tif")
files = glob("data/classic/smallim_*.tiff")
files = glob("data/buildings/smallim_*.tiff")
files = sorted(files)


# Read the images to be aligned
im1 = utils.readGTIFF(files[0]).squeeze()
h,w = im1.shape

gaussFiltSize = 1

imgs = []
transformations = []

for fname in files:
    # read the image 
    im2 = utils.readGTIFF(fname).squeeze()
    # align with the reference 
    im2_aligned, warp_matrix = ECC_frame_align(im1, im2, gaussFiltSize=gaussFiltSize)
    
    # append the result
    imgs.append(im2_aligned)
    transformations.append(warp_matrix)
    
    
vistools.display_gallery([ utils.simple_equalization_8bit(x) for x in  imgs ] )

# V. Super resolution of real satellite images 

* compute the image registration homographies 
* generate the sample list 
* call the fusion algorithm
* sharpen the result

In [ ]:
def convert_homography_into_samples(warp_matrix, shape):
    """
    apply the homography transformation (warp_matrix, 3x3) to the sample positions of an image of size (shape)
    returns the sample positions X, Y and a binary mask M  indicating which pixels fell outside of the image
    """
    h,w = shape

    # apply transformation to the samples 
    X, Y = np.meshgrid(np.arange(w), np.arange(h))
    X, Y = X.flatten(), Y.flatten()
    
    samplesloc = np.vstack(( X , Y, np.ones_like(X) ))
    newsamplesloc = np.linalg.inv(warp_matrix)@ samplesloc
    newsamplesloc[0,:]/=newsamplesloc[2,:]
    newsamplesloc[1,:]/=newsamplesloc[2,:]

    # determine mask 
    mask =  ((newsamplesloc[0,:]>0)  *  (newsamplesloc[0,:] <= w) *  (newsamplesloc[1,:] >0) * (newsamplesloc[1,:] <= h) )
    
    Xs = newsamplesloc[0,:].reshape(shape)
    Ys = newsamplesloc[1,:].reshape(shape)
    Ms = mask.reshape(shape)
    
    return Xs, Ys, Ms

In [ ]:
from glob import glob

files = glob("data/mire/crop*.tif")
files = glob("data/buildings/smallim_*.tiff")
files = glob("data/classic/smallim_*.tiff")
files = sorted(files)



# Read the images to be aligned
im1 = utils.readGTIFF(files[0]).squeeze()
h,w = im1.shape

gaussFiltSize = 5

warped = []
transformations = []

Xs=[] # sample x coordinate
Ys=[] # sample y coordinate
Vs=[] # sample value
Ms=[] # sample mask: true valid, false invalid

for fname in files:
    # read the image 
    im2 = utils.readGTIFF(fname).squeeze()
    # align with the reference 
    im2_aligned, warp_matrix = ECC_frame_align(im1, im2, gaussFiltSize=gaussFiltSize)

    # append the result
    warped.append(im2_aligned)
    transformations.append(warp_matrix)

    # apply transformation to the samples 
    X,Y,M = convert_homography_into_samples(warp_matrix, im1.shape)
     
    # concatenate with the samples 
    Xs.append(X)
    Ys.append(Y)
    Vs.append(im2)
    Ms.append(M)
    
Xs = np.array(Xs)
Ys = np.array(Ys)
Vs = np.array(Vs)
Ms = np.array(Ms)

In [ ]:
import matplotlib.pyplot as plt

vistools.display_gallery([ utils.simple_equalization_8bit(x) for x in  warped ] )

In [ ]:
h,w = Vs[0].shape
zz  = 2

u_sa  = shift_and_add( [int(h*zz),int(w*zz)], Vs, Xs*zz, Ys*zz, Ms, fill_missing=True)
u_act = ACT( [int(h*zz),int(w*zz)], Vs, Xs*zz, Ys*zz, Ms)

u_sa1  = sharpen(u_sa)
u_act1 = sharpen(u_act)

imshowfft(u_sa1,  title='S&A sharpened', z=zz, figsize=(20,20) )
imshowfft(u_act1, title='ACT sharpened', z=zz, figsize=(20,20))

vistools.display_gallery([ utils.simple_equalization_8bit(x) for x in [u_sa1, u_act1] ], ['S&A sharpened', 'ACT sharpened'])

#### Apply the above procedure to another senquence of images



> **<u>Excercise 3</u>**: Super-resolve a different set of images (from the ones provided).

In [ ]:
### EXERCISE. WRITE YOUR SOLUTION CODE HERE



### EXERCISE. WRITE YOUR SOLUTION CODE HERE

# BEGIN SOLUTION


from glob import glob

files = glob("data/mire/crop*.tif")
files = glob("data/buildings/smallim_*.tiff")
files = glob("data/classic/smallim_*.tiff")
files = sorted(files)


def align_frames(files, gaussFiltSize=5):

    # Read the images to be aligned
    im1 = utils.readGTIFF(files[0]).squeeze()
    h,w = im1.shape

    gaussFiltSize = 5

    warped = []
    transformations = []

    Xs=[] # sample x coordinate
    Ys=[] # sample y coordinate
    Vs=[] # sample value
    Ms=[] # sample mask: true valid, false invalid

    for fname in files:
        # read the image 
        im2 = utils.readGTIFF(fname).squeeze()
        # align with the reference 
        im2_aligned, warp_matrix = ECC_frame_align(im1, im2, gaussFiltSize=gaussFiltSize)
            
        # append the result
        warped.append(im2_aligned)
        transformations.append(warp_matrix)

        # apply transformation to the samples 
        X,Y,M = convert_homography_into_samples(warp_matrix, im1.shape)
        
        # concatenate with the samples 
        Xs.append(X)
        Ys.append(Y)
        Vs.append(im2)
        Ms.append(M)
        
    Xs = np.array(Xs)
    Ys = np.array(Ys)
    Vs = np.array(Vs)
    Ms = np.array(Ms)

    return Xs,Ys,Vs,Ms,warped 


Xs,Ys,Vs,Ms,warped  = align_frames(files, gaussFiltSize=5)

vistools.display_gallery([ utils.simple_equalization_8bit(x) for x in  warped ] )


h,w = Vs[0].shape
zz  = 2

u_sa  = shift_and_add( [int(h*zz),int(w*zz)], Vs, Xs*zz, Ys*zz, Ms, fill_missing=True)
u_act = ACT( [int(h*zz),int(w*zz)], Vs, Xs*zz, Ys*zz, Ms)

u_sa1  = sharpen(u_sa)
u_act1 = sharpen(u_act)

imshowfft(u_sa1,  title='S&A sharpened', z=zz, figsize=(20,20) )
imshowfft(u_act1, title='ACT sharpened', z=zz, figsize=(20,20))

vistools.display_gallery([ utils.simple_equalization_8bit(x) for x in [u_sa1, u_act1] ], ['S&A sharpened', 'ACT sharpened'])

# END SOLUTION

# Experiments

Here we study how the presented methods behave with different number of input images,  in presence of jitter (error in the sample positions), and  image noise.

In [ ]:
def RMSE(A,B,pad=20):
    return np.sqrt( ((A[pad:-pad,pad:-pad] - B[pad:-pad,pad:-pad])**2).mean())

> **<u>Exercise 4:</u>** analyze the impact of alignment errors (jitter) in the restorations for S&A and ACT methods. Plot the RMSE vs jitter error.

In [ ]:
### EXERCISE. WRITE YOUR SOLUTION CODE HERE


# BEGIN SOLUTION
sh = u.shape
H_sensor = sensor_OTF(sh,z,photodiode_ratio=photodiode_ratio)
H_diffraction =  diffraction_OTF(sh,D,f,lam,pixel_pitch,z)
H = H_sensor * H_diffraction

u_bl = np.real(ifft2(fft2(u) * H))

outs_sa = []
rmse_sa = []
outs_act = []
rmse_act = []
jitters  = 10** (np.linspace(-4,1,20))
for jj in jitters:
    
    Vs,Xs,Ys,Ms = simulate_LR_stack(u, H, 20, z = 2, sigma=0, jitter=jj)
    
    h,w = Vs[0].shape
    
    u_act = ACT( [int(h*z),int(w*z)], Vs, Xs*z, Ys*z,Ms)
    u_sa = shift_and_add( [int(h*z),int(w*z)], Vs, Xs*z, Ys*z,Ms)

    outs_act.append(u_act)
    outs_sa.append(u_sa)

    rmse_act.append( RMSE (u_bl, u_act) )
    rmse_sa.append( RMSE (u_bl, u_sa))

    
plt.loglog(jitters, rmse_act,label='act')
plt.loglog(jitters, rmse_sa,label='sa')
plt.legend()

vistools.display_gallery(outs_act)
# END SOLUTION

> **<u>Exercise 5:</u>** analyze the impact of noise in the restoration. Plot RMSE vs sigma of noise.

In [ ]:
### EXERCISE. WRITE YOUR SOLUTION CODE HERE


> **<u>Exercise 6:</u>** analyze the effect of having more images in the restoration. Plot RMSE vs number of images (e.g. `np.range(2,30,3)`) for the methods S&A and ACT and for different sigma levels (e.g. $\sigma=$ 0.3 and 1).

In [ ]:
### EXERCISE. WRITE YOUR SOLUTION CODE HERE



---------------------------
[//]: # (© 2021 Gabriele Facciolo)
[//]: # (<div style="text-align:center; font-size:75%;"> Copyright © 2021 Gabriele Facciolo. All rights reserved.</div> )